In [1]:
import pandas as pd

In [2]:
sez = pd.read_csv("joined_sez_map.csv")

In [3]:
pop = pd.read_csv("/Users/hendrixperalta/Desktop/Research Data Manipulation/research_data/adm1/population/p_landscan_pop.csv")

In [4]:
pop.rename(columns = {"asdf_id" : "id"}, inplace=True)

In [5]:
column_filter = pop[pop.columns[pop.columns.str.contains("land")]]

In [6]:
for col in column_filter:
    new_name = "pop" + col.split(".", 1)[1]
    pop.rename(columns = {col : new_name}, inplace = True)

In [7]:
pop_sum = pop[pop.columns[pop.columns.str.contains("sum|id")]]

In [8]:
pop_sum.columns = pop_sum.columns.str.replace(".sum$", "", regex=True)

In [9]:
pop_sum.dtypes

id           int64
pop2000    float64
pop2001    float64
pop2002    float64
pop2003    float64
pop2004    float64
pop2005    float64
pop2006    float64
pop2007    float64
pop2008    float64
pop2009    float64
pop2010    float64
pop2011    float64
pop2012    float64
pop2013    float64
pop2014    float64
pop2015    float64
pop2016    float64
pop2017    float64
pop2018    float64
pop2019    float64
pop2020    float64
pop2021    float64
dtype: object

In [10]:
#prep = pd.read_csv("/Users/hendrixperalta/Desktop/Research Data Manipulation/research_data/adm2/precipitation/precipitation_00_22.csv")

In [11]:
#prep_filter = prep[prep.columns[prep.columns.str.contains("pre_yearly")]]

In [12]:
#for col in prep_filter: 
#    new_name = "prep" + col.split(".", 1)[1]
#    prep.rename(columns = {col : new_name}, inplace = True)

In [13]:
#prep.columns = prep.columns.str.replace(".mean$", "", regex=True)

In [14]:
#prep.rename(columns = {"asdf_id" : "id"}, inplace=True)

In [15]:
#prep = prep[prep.columns[prep.columns.str.contains("prep|id")]]

In [16]:
temp = pd.read_csv("/Users/hendrixperalta/Desktop/Research Data Manipulation/research_data/adm1/temperature/p_landsurface_temp.csv")

In [17]:
temp.rename(columns = {"asdf_id" : "id"}, inplace=True)

In [18]:
temp_filter = temp[temp.columns[temp.columns.str.contains("mod11c3_061")]]

In [19]:
temp_filter = temp_filter[temp_filter.columns[temp_filter.columns.str.contains(".mean.1")]]

In [20]:
for col in temp_filter: 
    new_name = "temp" + col.split(".", 1)[1]
    temp.rename(columns = {col : new_name}, inplace=True)

In [21]:
temp.columns = temp.columns.str.replace(".mean.1", "", regex=True)

In [22]:
temp = temp[temp.columns[temp.columns.str.contains("temp|id")]]

In [23]:
temp.columns

Index(['id', 'temp2000', 'temp2001', 'temp2002', 'temp2003', 'temp2004',
       'temp2005', 'temp2006', 'temp2007', 'temp2008', 'temp2009', 'temp2010',
       'temp2011', 'temp2012', 'temp2013', 'temp2014', 'temp2015', 'temp2016',
       'temp2017', 'temp2018', 'temp2019', 'temp2020', 'temp2021', 'temp2022',
       'temp2023'],
      dtype='object')

In [24]:
agri = pd.read_csv("/Users/hendrixperalta/Desktop/Research Data Manipulation/research_data/adm1/landcover/agriculture_land_wide_adm1.csv")

In [25]:
agri.rename(columns = {"provinceCode" : "id"}, inplace=True)

In [26]:
agri = agri[agri.columns[agri.columns.str.contains("agr|id")]]

In [27]:
agri.columns

Index(['id', 'agr2001', 'agr2002', 'agr2003', 'agr2004', 'agr2005', 'agr2006',
       'agr2007', 'agr2008', 'agr2009', 'agr2010', 'agr2011', 'agr2012',
       'agr2013', 'agr2014', 'agr2015', 'agr2016', 'agr2017', 'agr2018',
       'agr2019', 'agr2020', 'agr2021', 'agr2022'],
      dtype='object')

In [28]:
urba = pd.read_csv("/Users/hendrixperalta/Desktop/Research Data Manipulation/research_data/adm1/landcover/urban_land_wide_adm1.csv")

In [29]:
urba.rename(columns = {"provinceCode" : "id"}, inplace=True)

In [30]:
urba = urba[urba.columns[urba.columns.str.contains("urb|id")]]

In [31]:
urba.columns

Index(['id', 'urb2001', 'urb2002', 'urb2003', 'urb2004', 'urb2005', 'urb2006',
       'urb2007', 'urb2008', 'urb2009', 'urb2010', 'urb2011', 'urb2012',
       'urb2013', 'urb2014', 'urb2015', 'urb2016', 'urb2017', 'urb2018',
       'urb2019', 'urb2020', 'urb2021', 'urb2022'],
      dtype='object')

In [32]:
ntl = pd.read_csv("/Users/hendrixperalta/Desktop/Research Data Manipulation/research_data/adm1/ntl/wide_ntl_adm1_2000_2021.csv")

In [33]:
ntl = ntl[ntl.columns[ntl.columns.str.contains("ntl|id")]]

In [34]:
ntl.columns

Index(['id', 'ntl2000', 'ntl2001', 'ntl2002', 'ntl2003', 'ntl2004', 'ntl2005',
       'ntl2006', 'ntl2007', 'ntl2008', 'ntl2009', 'ntl2010', 'ntl2011',
       'ntl2012', 'ntl2013', 'ntl2014', 'ntl2015', 'ntl2016', 'ntl2017',
       'ntl2018', 'ntl2019', 'ntl2020', 'ntl2021', 'ntl2022'],
      dtype='object')

In [35]:
egdp = pd.read_csv("/Users/hendrixperalta/Desktop/Research Data Manipulation/research_data/adm1/ntl/egdp_sum_wide_00-16.csv")

In [36]:
egdp = egdp[egdp.columns[egdp.columns.str.contains("egdp|id")]]

In [37]:
egdp.columns

Index(['id', 'egdp2000', 'egdp2001', 'egdp2002', 'egdp2003', 'egdp2004',
       'egdp2005', 'egdp2006', 'egdp2007', 'egdp2008', 'egdp2009', 'egdp2010',
       'egdp2011', 'egdp2012', 'egdp2013', 'egdp2014', 'egdp2015', 'egdp2016'],
      dtype='object')

In [38]:
datasets = [temp, agri, urba, ntl, egdp]

In [39]:
municipality_data = pd.merge(sez, pop_sum[pop_sum.columns[pop_sum.columns.str.contains("pop|id")]], on="id", how="outer")

In [40]:
for dataset in datasets: 
    municipality_data = pd.merge(municipality_data, dataset, on="id", how= "outer")

In [41]:
filter_year = ["2017", "2018", "2018", "2019", "2020", "2021", "2022", "2023","1999"]

pattern = "|".join(filter_year)
municipality_data = municipality_data.drop(columns = municipality_data.columns[municipality_data.columns.str.contains(pattern)])

In [42]:
#municipality_data.to_csv("municipality_data.csv", index=False)

In [43]:
var_names = ["inv", "ob_f_","tec_f_","adm_f_","ob_m_","tec_m_","adm_m_",'ent', 'ele', "com", 
"inf", "wat", "tss", "buiilt", "ocu", "sal_op", "sal_tec", "pop", "prep", "temp", "agr_", "urb_", "ntl_"]

In [44]:
pattern2 = "|".join(var_names)

In [45]:
long_df = pd.wide_to_long(municipality_data, stubnames=var_names, i='id', j='year', sep='')


In [46]:
long_df = long_df[long_df.columns[long_df.columns.str.contains("shape|id|year")|long_df.columns.str.contains(pattern2)]]

In [47]:
#long_df =  long_df.drop(columns = "id.1")
long_df =  long_df.drop(columns = "sez_id")

In [48]:
long_df

,,shapeName,year,inv,ob_f_,tec_f_,adm_f_,ob_m_,tec_m_,adm_m_,ent,...,buiilt,ocu,sal_op,sal_tec,pop,prep,temp,agr_,urb_,ntl_
id,year,,,,,,,,,,,,,,,,,,,,,
0.0,2000,SantiagoSantiagoSantiagoSantiagoSantiagoSantia...,54172.0,3.157257e+08,25544.0,1941.0,982.0,26264.0,3330.0,1258.0,127.0,...,6143797.00,5595246.00,267.621481,547.277778,8.099659e+05,NaN,300.565424,NaN,NaN,NaN
1.0,2000,DuarteDuarte,4003.0,5.331860e+06,514.0,24.0,11.0,398.0,27.0,10.0,5.0,...,152007.00,107925.00,273.590000,787.500000,3.411766e+05,NaN,301.359654,NaN,NaN,NaN
2.0,2000,ElÃ­as PiÃ±a,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.000000,0.000000,7.681411e+04,NaN,302.439574,NaN,NaN,NaN
3.0,2000,Hato Mayor,1990.0,2.376975e+06,309.0,8.0,6.0,166.0,17.0,12.0,3.0,...,82000.00,66097.00,587.330000,1700.000000,8.037471e+04,NaN,301.368647,NaN,NaN,NaN
4.0,2000,La Vega,1987.0,4.501822e+07,5597.0,277.0,156.0,6720.0,526.0,204.0,30.0,...,1283873.00,1283873.00,716.770000,1608.100000,3.776130e+05,NaN,298.775821,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27.0,2016,MarÃ­a Trinidad SÃ¡nchez,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.000000,0.000000,1.532622e+05,NaN,300.668988,NaN,NaN,NaN
28.0,2016,Santo DomingoSanto DomingoSanto DomingoSanto D...,38054.0,1.056726e+09,12988.0,2452.0,1159.0,8325.0,3916.0,1122.0,108.0,...,6656768.68,6001185.62,2444.102105,4511.845789,2.627903e+06,NaN,302.331366,NaN,NaN,NaN
29.0,2016,Monte Plata,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.000000,0.000000,2.086535e+05,NaN,300.587101,NaN,NaN,NaN


In [49]:
long_df.dtypes

shapeName     object
year         float64
inv          float64
ob_f_        float64
tec_f_       float64
adm_f_       float64
ob_m_        float64
tec_m_       float64
adm_m_       float64
ent          float64
ele          float64
com          float64
inf          float64
wat          float64
tss          float64
buiilt       float64
ocu          float64
sal_op       float64
sal_tec      float64
pop          float64
prep         float64
temp         float64
agr_         float64
urb_         float64
ntl_         float64
dtype: object

In [50]:
long_df.to_csv("long_province_sez.csv")